# 机器翻译


## 背景介绍

机器翻译（machine translation, MT）是用计算机来实现不同语言之间翻译的技术。被翻译的语言通常称为源语言（source language），翻译成的结果语言称为目标语言（target language）。机器翻译即实现从源语言到目标语言转换的过程，是自然语言处理的重要研究领域之一。

早期机器翻译系统多为基于规则的翻译系统，需要由语言学家编写两种语言之间的转换规则，再将这些规则录入计算机。该方法对语言学家的要求非常高，而且我们几乎无法总结一门语言会用到的所有规则，更何况两种甚至更多的语言。因此，传统机器翻译方法面临的主要挑战是无法得到一个完备的规则集合\[[1](#参考文献)\]。

为解决以上问题，统计机器翻译（Statistical Machine Translation, SMT）技术应运而生。在统计机器翻译技术中，转化规则是由机器自动从大规模的语料中学习得到的，而非我们人主动提供规则。因此，它克服了基于规则的翻译系统所面临的知识获取瓶颈的问题，但仍然存在许多挑战：1）人为设计许多特征（feature），但永远无法覆盖所有的语言现象；2）难以利用全局的特征；3）依赖于许多预处理环节，如词语对齐、分词或符号化（tokenization）、规则抽取、句法分析等，而每个环节的错误会逐步累积，对翻译的影响也越来越大。

近年来，深度学习技术的发展为解决上述挑战提供了新的思路。将深度学习应用于机器翻译任务的方法大致分为两类：1）仍以统计机器翻译系统为框架，只是利用神经网络来改进其中的关键模块，如语言模型、调序模型等（见图1的左半部分）；2）不再以统计机器翻译系统为框架，而是直接用神经网络将源语言映射到目标语言，即端到端的神经网络机器翻译（End-to-End Neural Machine Translation, End-to-End NMT）（见图1的右半部分），简称为NMT模型。
![](https://ai-studio-static-online.cdn.bcebos.com/857aa804f8bb4bd2af46f517510ed667f371dba449544f50a440823f9b433436)
图1. 基于神经网络的机器翻译系统

本教程主要介绍NMT模型，以及如何用PaddlePaddle来训练一个NMT模型。

## 效果展示

以中英翻译（中文翻译到英文）的模型为例，当模型训练完毕时，如果输入如下已分词的中文句子：
```text
这些 是 希望 的 曙光 和 解脱 的 迹象 .
```
如果设定显示翻译结果的条数（即[柱搜索算法](#柱搜索算法)的宽度）为3，生成的英语句子如下：
```text
0 -5.36816   These are signs of hope and relief . <e>
1 -6.23177   These are the light of hope and relief . <e>
2 -7.7914  These are the light of hope and the relief of hope . <e>
```

- 左起第一列是生成句子的序号；左起第二列是该条句子的得分（从大到小），分值越高越好；左起第三列是生成的英语句子。

- 另外有两个特殊标志：`<e>`表示句子的结尾，`<unk>`表示未登录词（unknown word），即未在训练字典中出现的词。

## 模型概览

本节依次介绍双向循环神经网络（Bi-directional Recurrent Neural Network），NMT模型中典型的编码器-解码器（Encoder-Decoder）框架以及柱搜索（beam search）算法。

### 双向循环神经网络

这里介绍Bengio团队在论文\[[2](#参考文献),[4](#参考文献)\]中提出的一种双向循环神经网络结构（其他结构的双向RNN可参考[语义角色标注](https://github.com/PaddlePaddle/book/blob/develop/07.label_semantic_roles/README.cn.md)）。该结构的目的是输入一个序列，得到其在每个时刻的特征表示，即输出的每个时刻都用定长向量表示到该时刻的上下文语义信息。

具体来说，该双向循环神经网络分别在时间维以顺序和逆序——即前向（forward）和后向（backward）——依次处理输入序列，并将每个时间步RNN的输出拼接成为最终的输出层。这样每个时间步的输出节点，都包含了输入序列中当前时刻完整的过去和未来的上下文信息。下图展示的是一个按时间步展开的双向循环神经网络。该网络包含一个前向和一个后向RNN，其中有六个权重矩阵：输入到前向隐层和后向隐层的权重矩阵（$W_1, W_3$），隐层到隐层自己的权重矩阵（$W_2,W_5$），前向隐层和后向隐层到输出层的权重矩阵（$W_4, W_6$）。注意，该网络的前向隐层和后向隐层之间没有连接。


<div align="center">
<img src = "https://ai-studio-static-online.cdn.bcebos.com/cd49b095d5a1460d854fd1b1fb5e59c9b8986539d277480cab1d9368a1706bd8" width="400"><br/>
图2. 按时间步展开的双向循环神经网络
</div>

### GRU

GRU[2]是Cho等人在LSTM上提出的简化版本，也是RNN的一种扩展，如下图所示。GRU单元只有两个门：

重置门（reset gate）：如果重置门关闭，会忽略掉历史信息，即历史不相干的信息不会影响未来的输出。
更新门（update gate）：将LSTM的输入门和遗忘门合并，用于控制历史信息对当前时刻隐层输出的影响。如果更新门接近1，会把历史信息传递下去。
<div align="center">
<img src = "https://ai-studio-static-online.cdn.bcebos.com/2e7f0789a63b44bcbbb590d15b718de9376a2b0478624019b7fc2cdb32b55ae0" width="400"><br/>
图3. GRU（门控循环单元）
</div>
一般来说，具有短距离依赖属性的序列，其重置门比较活跃；相反，具有长距离依赖属性的序列，其更新门比较活跃。另外，Chung等人[3]通过多组实验表明，GRU虽然参数更少，但是在多个任务上都和LSTM有相近的表现。


### 编码器-解码器框架

编码器-解码器（Encoder-Decoder）\[[2](#参考文献)\]框架用于解决由一个任意长度的源序列到另一个任意长度的目标序列的变换问题。即编码阶段将整个源序列编码成一个向量，解码阶段通过最大化预测序列概率，从中解码出整个目标序列。编码和解码的过程通常都使用RNN实现。
<div align="center">
<img src ="https://ai-studio-static-online.cdn.bcebos.com/6e7106ca8e584d4c802f6577c34a3501b49ac94ad31040e2a1f73817fb8fc83d" width="500"><br/>
图4. 编码器-解码器框架
</div>

<a name="编码器"></a>
#### 编码器

编码阶段分为三步：

1. one-hot vector表示：将源语言句子$x=\left \{ x_1,x_2,...,x_T \right \}$的每个词$x_i$表示成一个列向量$w_i\epsilon \left \{ 0,1 \right \}^{\left | V \right |},i=1,2,...,T$。这个向量$w_i$的维度与词汇表大小$\left | V \right |$ 相同，并且只有一个维度上有值1（该位置对应该词在词汇表中的位置），其余全是0。

2. 映射到低维语义空间的词向量：one-hot vector表示存在两个问题，1）生成的向量维度往往很大，容易造成维数灾难；2）难以刻画词与词之间的关系（如语义相似性，也就是无法很好地表达语义）。因此，需再one-hot vector映射到低维的语义空间，由一个固定维度的稠密向量（称为词向量）表示。记映射矩阵为$C\epsilon R^{K\times \left | V \right |}$，用$s_i=Cw_i$表示第$i$个词的词向量，$K$为向量维度。

3. 用RNN编码源语言词序列：这一过程的计算公式为$h_i=\varnothing _\theta \left ( h_{i-1}, s_i \right )$，其中$h_0$是一个全零的向量，$\varnothing _\theta$是一个非线性激活函数，最后得到的$\mathbf{h}=\left \{ h_1,..., h_T \right \}$就是RNN依次读入源语言$T$个词的状态编码序列。整句话的向量表示可以采用$\mathbf{h}$在最后一个时间步$T$的状态编码，或使用时间维上的池化（pooling）结果。

第3步也可以使用双向循环神经网络实现更复杂的句编码表示，具体可以用双向GRU实现。前向GRU按照词序列$(x_1,x_2,...,x_T)$的顺序依次编码源语言端词，并得到一系列隐层状态![](https://ai-studio-static-online.cdn.bcebos.com/3127aa9c68d44cdcad62ca9a92068a453c6bb7eeccfd4cbab08d399584d09a99)。类似的，后向GRU按照$(x_T,x_{T-1},...,x_1)$的顺序依次编码源语言端词，得到![](https://ai-studio-static-online.cdn.bcebos.com/721a6cbda9364649b207d03eb3d4741630700bc4fe0341e6b88e8936a25e2a3c)。最后对于词$x_i$，通过拼接两个GRU的结果得到它的隐层状态，即![](https://ai-studio-static-online.cdn.bcebos.com/aa1f71a19f224eac8bda10e3692431f794bfbf5d28754506bdaa9ce00de6a0d8)。

<div align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/a6d7b58a6fab450e89b706106cca0f5d70f0c6a4f2504e44a0063e0f4afac119" width="400"><br/>
图5. 使用双向GRU的编码器
</div>

#### 解码器

机器翻译任务的训练过程中，解码阶段的目标是最大化下一个正确的目标语言词的概率。思路是：
1. 每一个时刻，根据源语言句子的编码信息（又叫上下文向量，context vector）$c$、真实目标语言序列的第$i$个词$u_i$和$i$时刻RNN的隐层状态$z_i$，计算出下一个隐层状态$z_{i+1}$。计算公式如下：
$$z_{i+1}=\phi_{\theta '} \left ( c,u_i,z_i \right )$$
其中$\phi _{\theta '}$是一个非线性激活函数；$c$是源语言句子的上下文向量，在不使用注意力机制时，如果[编码器](#编码器)的输出是源语言句子编码后的最后一个元素，则可以定义$c=h_T$；$u_i$是目标语言序列的第$i$个单词，$u_0$是目标语言序列的开始标记`<s>`，表示解码开始；$z_i$是$i$时刻解码RNN的隐层状态，$z_0$是一个全零的向量。

2. 将$z_{i+1}$通过`softmax`归一化，得到目标语言序列的第$i+1$个单词的概率分布$p_{i+1}$。概率分布公式如下：
![](https://ai-studio-static-online.cdn.bcebos.com/0ba96afb4ed442aa838a514966b969ea65fee1f8057f4a42a3a4f6ec12295755)

其中$W_sz_{i+1}+b_z$是对每个可能的输出单词进行打分，再用softmax归一化就可以得到第$i+1$个词的概率$p_{i+1}$。

3. 根据$p_{i+1}$和$u_{i+1}$计算代价。

4. 重复步骤1~3，直到目标语言序列中的所有词处理完毕。

机器翻译任务的生成过程，通俗来讲就是根据预先训练的模型来翻译源语言句子。生成过程中的解码阶段和上述训练过程的有所差异，具体介绍请见[柱搜索算法](#柱搜索算法)。

<a name="柱搜索算法"></a>

### 柱搜索算法

柱搜索（[beam search](http://en.wikipedia.org/wiki/Beam_search)）是一种启发式图搜索算法，用于在图或树中搜索有限集合中的最优扩展节点，通常用在解空间非常大的系统（如机器翻译、语音识别）中，原因是内存无法装下图或树中所有展开的解。如在机器翻译任务中希望翻译“`<s>你好<e>`”，就算目标语言字典中只有3个词（`<s>`, `<e>`, `hello`），也可能生成无限句话（`hello`循环出现的次数不定），为了找到其中较好的翻译结果，我们可采用柱搜索算法。

柱搜索算法使用广度优先策略建立搜索树，在树的每一层，按照启发代价（heuristic cost）（本教程中，为生成词的log概率之和）对节点进行排序，然后仅留下预先确定的个数（文献中通常称为beam width、beam size、柱宽度等）的节点。只有这些节点会在下一层继续扩展，其他节点就被剪掉了，也就是说保留了质量较高的节点，剪枝了质量较差的节点。因此，搜索所占用的空间和时间大幅减少，但缺点是无法保证一定获得最优解。

使用柱搜索算法的解码阶段，目标是最大化生成序列的概率。思路是：
1. 每一个时刻，根据源语言句子的编码信息$c$、生成的第$i$个目标语言序列单词$u_i$和$i$时刻RNN的隐层状态$z_i$，计算出下一个隐层状态$z_{i+1}$。

2. 将$z_{i+1}$通过`softmax`归一化，得到目标语言序列的第$i+1$个单词的概率分布$p_{i+1}$。

3. 根据$p_{i+1}$采样出单词$u_{i+1}$。

4. 重复步骤1~3，直到获得句子结束标记`<e>`或超过句子的最大生成长度为止。

注意：$z_{i+1}$和$p_{i+1}$的计算公式同[解码器](#解码器)中的一样。且由于生成时的每一步都是通过贪心法实现的，因此并不能保证得到全局最优解。

假设字典为[a,b,c]，beam size选择2，则如下图有：

1、在生成第1个词的时候，选择概率最大的2个词，那么当前序列就是a或b;

2、生成第2个词的时候，我们将当前序列a或b，分别与字典中的所有词进行组合，得到新的6个序列aa ab ac ba bb bc,然后从其中选择2个概率最高的，作为当前序列，即ab或bb;

3、不断重复这个过程，直到遇到结束符为止。最终输出2个概率最高的序列。

![](https://ai-studio-static-online.cdn.bcebos.com/c78c01d4a57940848e135173a706d374fa5411ff3d2945e4958e40f4ef0ffa94)

## 数据介绍

本教程使用WMT-16新增的multimodal task中的translation task的数据集。该数据集为英德翻译数据，包含29001条训练数据，1000条测试数据。

### 数据预处理

我们的预处理流程包括两步：

- 将每个源语言到目标语言的平行语料库文件合并为一个文件：

- 合并每个`XXX.src`和`XXX.trg`文件为`XXX`。

- `XXX`中的第$i$行内容为`XXX.src`中的第$i$行和`XXX.trg`中的第$i$行连接，用'\t'分隔。

- 创建训练数据的“源字典”和“目标字典”。每个字典都有**DICTSIZE**个单词，包括：语料中词频最高的（DICTSIZE - 3）个单词，和3个特殊符号`<s>`（序列的开始）、`<e>`（序列的结束）和`<unk>`（未登录词）。

### 示例数据

为了验证训练流程，PaddlePaddle接口paddle.dataset.wmt16中提供了对该数据集预处理后的版本，调用该接口即可直接使用，因为数据规模限制，这里只作为示例使用，在相应的测试集上具有一定效果但在更多测试数据上的效果无法保证。


## 模型配置说明

下面我们开始根据输入数据的形式配置模型。首先引入所需的库函数以及定义全局变量。

In [8]:
import paddle.fluid as fluid
import numpy as np
in1 = np.array([[1,2,3],
                [4,5,6]])
in2 = np.array([[11,12,13],
                [14,15,16]])
in3 = np.array([[21,22],
                [23,24]])
with fluid.dygraph.guard():
    x1 = fluid.dygraph.to_variable(in1)
    x2 = fluid.dygraph.to_variable(in2)
    x3 = fluid.dygraph.to_variable(in3)
    out1 = fluid.layers.concat(input=[x1,x2,x3], axis=1) # 向量拼接，在二个方向上面拼接
    out2 = fluid.layers.concat(input=[x1,x2], axis=1)
    print(out1)
    print(out1.numpy())
    print(out2.numpy())

Tensor(shape=[2, 8], dtype=int32, place=CPUPlace, stop_gradient=True,
       [[1 , 2 , 3 , 11, 12, 13, 21, 22],
        [4 , 5 , 6 , 14, 15, 16, 23, 24]])
[[ 1  2  3 11 12 13 21 22]
 [ 4  5  6 14 15 16 23 24]]
[[ 1  2  3 11 12 13]
 [ 4  5  6 14 15 16]]


In [2]:
import paddle
train_data = paddle.batch(
    paddle.reader.shuffle(
        paddle.dataset.wmt16.train(30000, 30000),
        buf_size=10000),
    batch_size=64)
sample_data = next(train_data())
print(sample_data[0])

C:\Users\Jaon\AppData\Local\Temp/ipykernel_13640/207610793.py:4: DeprecationWarning: 
API "paddle.dataset.wmt16.train" is deprecated since 2.0.0, and will be removed in future versions. Please use "paddle.text.datasets.WMT16" instead.
reason: Please use new dataset API which supports paddle.io.DataLoader 
  paddle.dataset.wmt16.train(30000, 30000),
Cache file C:\Users\Jaon\.cache\paddle\dataset\wmt16\wmt16.tar.gz not found, downloading http://paddlemodels.bj.bcebos.com/wmt/wmt16.tar.gz 
Begin to download

Download finished


([0, 5, 11, 112, 109, 392, 3, 451, 8, 7, 87, 4, 1], [0, 4, 11, 15, 755, 169, 17, 556, 10, 25, 503, 3], [4, 11, 15, 755, 169, 17, 556, 10, 25, 503, 3, 1])


In [3]:
# 导入包
import os

import numpy as np
import paddle
import paddle.fluid as fluid

In [4]:
dict_size = 30000  # 词典大小
source_dict_size = target_dict_size = dict_size  # 源/目标语言字典大小

word_dim = 512  # 词向量维度 （就是参数向量的行）
hidden_dim = 512  # 编码器中的隐层大小 （其实就是权重向量中的行）
decoder_size = hidden_dim  # 解码器中的隐层大小 （其实就是权重向量中的行）

max_length = 256  # 解码生成句子的最大长度（柱搜索算法的截止点）
# 下面确定了每次检索，留下4个词，其余的全部剪枝，这四个也是概率最大的（log大的）
beam_size = 4  # beam search的柱宽度 按照启发代价（heuristic cost）（本教程中，为生成词的log概率之和）对节点进行排序，然后仅留下预先确定的个数（文献中通常称为beam width、beam size、柱宽度等）的节点。

batch_size = 64  # batch 中的样本数
is_sparse = True
model_save_dir = "machine_translation.inference.model"

In [5]:
#构建编码器
def encoder():
     # 定义源语言id序列的输入数据
    src_word_id = fluid.layers.data(
        name="src_word_id", shape=[1], dtype='int64', lod_level=1)

    # 将上述编码映射到低维语言空间的词向量
    src_embedding = fluid.layers.embedding(
        input=src_word_id,
        size=[source_dict_size, word_dim],
        dtype='float32',
        is_sparse=is_sparse)

    # 用双向GRU编码源语言序列，拼接两个GRU的编码结果得到h
    # 以embeding层的输出作为输入，建立全连接层，全连接层的节点数为：GRN中隐层的个数的三倍
    fc_forward = fluid.layers.fc(
        input=src_embedding, size=hidden_dim * 3, bias_attr=False)
    src_forward = fluid.layers.dynamic_gru(input=fc_forward, size=hidden_dim)
    # 这一层返回的张量的形状 (-1,512)，（就单个而言，也可以理解为512维的向量）
    print('the shape of the forward gru: ',src_forward.shape)

    # 后向隐层
    fc_backward = fluid.layers.fc(
        input=src_embedding, size=hidden_dim * 3, bias_attr=False)
    src_backward = fluid.layers.dynamic_gru(
        input=fc_backward, size=hidden_dim, is_reverse=True)
    # 这一层返回的向量的形状 （-1,512）
    print('the shape of the backward gru: ',src_backward.shape)
    
    # 将前向和后向进行拼接，作为输出
    # axis =1 表示在第二个方向上拼接，这一层应该是(-1,1014),也就是单词序列
    encoded_vector = fluid.layers.concat(
        input=[src_forward, src_backward], axis=1)
    print('the shape of encoded_vector : ',encoded_vector.shape)
    return encoded_vector

In [ ]:
# 定义解码过程RNN中的单步计算
def cell(x, hidden, encoder_out, encoder_out_proj):
    # 定义attention用以计算context，即 c_i，这里使用Bahdanau attention机制，
    # 使用注意力机制来计算context信息，也就是c
    def simple_attention(encoder_out, encoder_proj, decoder_state): # 使用注意力机制，计算context
        # 建立一个全连接层，入参为上一个节点的hidden
        decoder_state_proj = fluid.layers.fc(
            input=decoder_state, size=decoder_size, bias_attr=False)

        # sequence_expand将单步内容扩展为与encoder输出相同的LODTensor
        decoder_state_expand = fluid.layers.sequence_expand(
            x=decoder_state_proj, y=encoder_proj)
        mixed_state = fluid.layers.elementwise_add(encoder_proj,
                                                   decoder_state_expand)
                                                   
        attention_weights = fluid.layers.fc(
            input=mixed_state, size=1, bias_attr=False)
        attention_weights = fluid.layers.sequence_softmax(
            input=attention_weights)
        weigths_reshape = fluid.layers.reshape(x=attention_weights, shape=[-1])
        #print(weigths_reshape.shape)
        scaled = fluid.layers.elementwise_mul(
            x=encoder_out, y=weigths_reshape, axis=0)
        context = fluid.layers.sequence_pool(input=scaled, pool_type='sum')
        return context
    #cell(x, pre_state, encoder_out, encoder_out_proj)
    # 基于attention机制来计算c的时候，需要编码器的输出、编码器输出的概率、上一个节点的隐层状态(h)
    context = simple_attention(encoder_out, encoder_out_proj, hidden)
    out = fluid.layers.fc(
        input=[x, context], size=decoder_size * 3, bias_attr=False)
    out = fluid.layers.gru_unit(
        input=out, hidden=hidden, size=decoder_size * 3)[0]
    return out, out

In [ ]:
# 构建训练用的解码器
def train_decoder(encoder_out):
    # 获取编码器输出的最后一步并进行非线性映射以构造解码器RNN的初始状态
    # ecoder_out是一个包含n个序列，每个序列长度为1024的LoDTensor
    encoder_last = fluid.layers.sequence_last_step(input=encoder_out)
    # 这里返回的shape应该是(-1,1)前者为样本量的个数
    print('encoder_last shape : ',encoder_last.shape)
    # 创建一个全连接层，节点的个数为解码器的隐层的个数，这里使用了tanh函数，可以做归一化处理，
    # 可理解为针对编码的信息，计算出来的概率
    encoder_last_proj = fluid.layers.fc(
        input=encoder_last, size=decoder_size, act='tanh')
    
    # 编码器输出在attention中计算结果的cache
    # 暂时没看懂一步的概率
    encoder_out_proj = fluid.layers.fc(
        input=encoder_out, size=decoder_size, bias_attr=False)
    # 上面的内容都是在对编码器的输出做处理，处理完毕后，作为解码器的输入

    # 定义目标语言id序列的输入数据，并映射到低维语言空间的词向量
    trg_language_word = fluid.layers.data(
        name="target_language_word", shape=[1], dtype='int64', lod_level=1)
    trg_embedding = fluid.layers.embedding(
        input=trg_language_word,
        size=[target_dict_size, word_dim],
        dtype='float32',
        is_sparse=is_sparse)

    # 创建解码的rnn
    rnn = fluid.layers.DynamicRNN()
    with rnn.block():
        # 获取当前步目标语言输入的词向量
        x = rnn.step_input(trg_embedding)
        # 获取隐层状态
        pre_state = rnn.memory(init=encoder_last_proj, need_reorder=True)
        # 在DynamicRNN中需使用static_input获取encoder相关的内容
        # 对decoder来说这些内容在每个时间步都是固定的
        encoder_out = rnn.static_input(encoder_out)
        encoder_out_proj = rnn.static_input(encoder_out_proj)
        # 执行单步的计算单元
        # 以当前的单词、隐状态、c作为输入（c使用encoder_out and encoder_out_proj作为输入）
        # 计算下一个隐状态
        out, current_state = cell(x, pre_state, encoder_out, encoder_out_proj)
        # 计算归一化的单词预测概率
        # 到目标语言序列的第i+1i+1i+1个单词的概率分布pi+1
        prob = fluid.layers.fc(input=out, size=target_dict_size, act='softmax')
        # 更新隐层状态
        rnn.update_memory(pre_state, current_state)
        # 输出预测概率
        rnn.output(prob)

    return rnn()

In [6]:


#构建训练模型及参数
def train_model():
    # 创建编码器
    encoder_out = encoder()
    # 创建解码器，使用编码器的结果作为输入
    rnn_out = train_decoder(encoder_out)
    label = fluid.layers.data(
        name="target_language_next_word", shape=[1], dtype='int64', lod_level=1)
    cost = fluid.layers.cross_entropy(input=rnn_out, label=label)
    avg_cost = fluid.layers.mean(cost)
    return avg_cost

#构建训练过程优化参数
def optimizer_func():
    # 设置梯度裁剪
    fluid.clip.set_gradient_clip(
        clip=fluid.clip.GradientClipByGlobalNorm(clip_norm=5.0))
    # 定义先增后降的学习率策略
    lr_decay = fluid.layers.learning_rate_scheduler.noam_decay(hidden_dim, 1000)
    return fluid.optimizer.Adam(
        learning_rate=lr_decay,
        regularization=fluid.regularizer.L2DecayRegularizer(
            regularization_coeff=1e-4))

#训练过程
def train(use_cuda):
    train_prog = fluid.Program()
    startup_prog = fluid.Program()
    
    with fluid.program_guard(train_prog, startup_prog):
        with fluid.unique_name.guard():
            avg_cost = train_model()
            optimizer = optimizer_func()
            optimizer.minimize(avg_cost)

    place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
    exe = fluid.Executor(place)

    train_data = paddle.batch(
        paddle.reader.shuffle(
            paddle.dataset.wmt16.train(source_dict_size, target_dict_size),
            buf_size=10000),
        batch_size=batch_size)

    feeder = fluid.DataFeeder(
        feed_list=[
            'src_word_id', 'target_language_word', 'target_language_next_word'
        ],
        place=place,
        program=train_prog)

    exe.run(startup_prog)

    EPOCH_NUM = 2
    for pass_id in range(EPOCH_NUM):
        for batch_id,data in enumerate(train_data()):
            cost = exe.run(
                train_prog, feed=feeder.feed(data), fetch_list=[avg_cost])[0]
            if ( batch_id % 100 == 0):
                print('pass_id: %d, batch_id: %d, loss: %f' % (pass_id, batch_id,cost))
        fluid.io.save_params(exe, model_save_dir, main_program=train_prog)
    print ('save models to %s' % (model_save_dir))

#构建测试用的解码器（柱搜索）
def infer_decoder(encoder_out):
    # 获取编码器输出的最后一步并进行非线性映射以构造解码器RNN的初始状态
    encoder_last = fluid.layers.sequence_last_step(input=encoder_out)
    encoder_last_proj = fluid.layers.fc(
        input=encoder_last, size=decoder_size, act='tanh')
    # 编码器输出在attention中计算结果的cache
    encoder_out_proj = fluid.layers.fc(
        input=encoder_out, size=decoder_size, bias_attr=False)

    # 最大解码步数
    max_len = fluid.layers.fill_constant(
        shape=[1], dtype='int64', value=max_length)
    # 解码步数计数变量
    counter = fluid.layers.zeros(shape=[1], dtype='int64', force_cpu=True)

    # 定义 tensor array 用以保存各个时间步的内容，并写入初始id，score和state
    init_ids = fluid.layers.data(
        name="init_ids", shape=[1], dtype="int64", lod_level=2)
    init_scores = fluid.layers.data(
        name="init_scores", shape=[1], dtype="float32", lod_level=2)
    ids_array = fluid.layers.array_write(init_ids, i=counter)
    scores_array = fluid.layers.array_write(init_scores, i=counter)
    state_array = fluid.layers.array_write(encoder_last_proj, i=counter)

    # 定义循环终止条件变量
    cond = fluid.layers.less_than(x=counter, y=max_len)
    while_op = fluid.layers.While(cond=cond)
    with while_op.block():#单次循环内容
        # 获取解码器在当前步的输入，包括上一步选择的id，对应的score和上一步的state
        pre_ids = fluid.layers.array_read(array=ids_array, i=counter)
        pre_score = fluid.layers.array_read(array=scores_array, i=counter)
        pre_state = fluid.layers.array_read(array=state_array, i=counter)

        # 同train_decoder中的内容，进行RNN的单步计算
        pre_ids_emb = fluid.layers.embedding(
            input=pre_ids,
            size=[target_dict_size, word_dim],
            dtype='float32',
            is_sparse=is_sparse)
        out, current_state = cell(pre_ids_emb, pre_state, encoder_out,
                            encoder_out_proj)
        prob = fluid.layers.fc(
            input=current_state, size=target_dict_size, act='softmax')

        # 计算累计得分，进行beam search
        topk_scores, topk_indices = fluid.layers.topk(prob, k=beam_size)
        accu_scores = fluid.layers.elementwise_add(
            x=fluid.layers.log(topk_scores),
            y=fluid.layers.reshape(pre_score, shape=[-1]),
            axis=0)
        accu_scores = fluid.layers.lod_reset(x=accu_scores, y=pre_ids)#为打分结果添加LOD信息
        selected_ids, selected_scores = fluid.layers.beam_search(
            pre_ids, pre_score, topk_indices, accu_scores, beam_size, end_id=1)

        fluid.layers.increment(x=counter, value=1, in_place=True)#将张量counter的值加一
        # 将 search 结果写入 tensor array 中
        fluid.layers.array_write(selected_ids, array=ids_array, i=counter)
        fluid.layers.array_write(selected_scores, array=scores_array, i=counter)
        # sequence_expand 作为 gather 使用以获取search结果对应的状态，并更新
        current_state = fluid.layers.sequence_expand(current_state,
                                                     selected_ids)
        fluid.layers.array_write(current_state, array=state_array, i=counter)
        current_enc_out = fluid.layers.sequence_expand(encoder_out,
                                                       selected_ids)
        fluid.layers.assign(current_enc_out, encoder_out)
        current_enc_out_proj = fluid.layers.sequence_expand(
            encoder_out_proj, selected_ids)
        fluid.layers.assign(current_enc_out_proj, encoder_out_proj)

        # 更新循环终止条件
        length_cond = fluid.layers.less_than(x=counter, y=max_len)
        finish_cond = fluid.layers.logical_not(
            fluid.layers.is_empty(x=selected_ids))
        fluid.layers.logical_and(x=length_cond, y=finish_cond, out=cond)

    # 根据保存的每一步的结果，回溯生成最终解码结果
    translation_ids, translation_scores = fluid.layers.beam_search_decode(
        ids=ids_array, scores=scores_array, beam_size=beam_size, end_id=1)

    return translation_ids, translation_scores

#构建预测模型
def infer_model():
    encoder_out = encoder()
    translation_ids, translation_scores = infer_decoder(encoder_out)
    return translation_ids, translation_scores

#预测过程
def infer(use_cuda):
    infer_prog = fluid.Program()
    startup_prog = fluid.Program()
    with fluid.program_guard(infer_prog, startup_prog):
        with fluid.unique_name.guard():
            translation_ids, translation_scores = infer_model()

    place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()
    exe = fluid.Executor(place)

    test_data = paddle.batch(
        paddle.dataset.wmt16.test(source_dict_size, target_dict_size),
        batch_size=10)
    src_idx2word = paddle.dataset.wmt16.get_dict(
        "en", source_dict_size, reverse=True)
    trg_idx2word = paddle.dataset.wmt16.get_dict(
        "de", target_dict_size, reverse=True)

    fluid.io.load_params(exe, model_save_dir, main_program=infer_prog)
    for data in test_data():
        src_word_id = fluid.create_lod_tensor(
            data=[x[0] for x in data],
            recursive_seq_lens=[[len(x[0]) for x in data]],
            place=place)
        init_ids = fluid.create_lod_tensor(
            data=np.array([[0]] * len(data), dtype='int64'),
            recursive_seq_lens=[[1] * len(data)] * 2,
            place=place)
        init_scores = fluid.create_lod_tensor(
            data=np.array([[0.]] * len(data), dtype='float32'),
            recursive_seq_lens=[[1] * len(data)] * 2,
            place=place)
        seq_ids, seq_scores = exe.run(
            infer_prog,
            feed={
                'src_word_id': src_word_id,
                'init_ids': init_ids,
                'init_scores': init_scores
            },
            fetch_list=[translation_ids, translation_scores],
            return_numpy=False)
        # How to parse the results:
        #   Suppose the lod of seq_ids is:
        #     [[0, 3, 6], [0, 12, 24, 40, 54, 67, 82]]
        #   then from lod[0]:
        #     there are 2 source sentences, beam width is 3.
        #   from lod[1]:
        #     the first source sentence has 3 hyps; the lengths are 12, 12, 16
        #     the second source sentence has 3 hyps; the lengths are 14, 13, 15
        hyps = [[] for i in range(len(seq_ids.lod()[0]) - 1)]
        scores = [[] for i in range(len(seq_scores.lod()[0]) - 1)]
        for i in range(len(seq_ids.lod()[0]) - 1):  # for each source sentence
            start = seq_ids.lod()[0][i]
            end = seq_ids.lod()[0][i + 1]
            print("Original sentence:")
            print(" ".join([src_idx2word[idx] for idx in data[i][0][1:-1]]))
            print("Translated score and sentence:")
            for j in range(end - start):  # for each candidate
                sub_start = seq_ids.lod()[1][start + j]
                sub_end = seq_ids.lod()[1][start + j + 1]
                hyps[i].append(" ".join([
                    trg_idx2word[idx]
                    for idx in np.array(seq_ids)[sub_start:sub_end][1:-1]
                ]))
                scores[i].append(np.array(seq_scores)[sub_end - 1])
                print(scores[i][-1], hyps[i][-1].encode('utf8'))
        break


In [ ]:
def main(use_cuda):
    train(use_cuda)
    infer(use_cuda)


if __name__ == '__main__':
    use_cuda = False  # set to True if training with GPU
    main(use_cuda)